In [18]:
import time
import requests
import base64
import pandas as pd
from collections import deque

Data Extraction

In [19]:
# Your Spotify API credentials
CLIENT_ID = '3c5805d570ac43279a5585c95db31a11'
CLIENT_SECRET = '1248b4671b284effbe92c559c4f51bec'

# Base64 encode the client_id and client_secret
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode("utf-8")

# Define the URL for Spotify's token endpoint
TOKEN_URL = "https://accounts.spotify.com/api/token"

# Prepare the data for the POST request
headers = {
    "Authorization": f"Basic {auth_header}"
}

data = {
    "grant_type": "client_credentials"
}

# Make the POST request to get the token
response = requests.post(TOKEN_URL, headers=headers, data=data)

# Get the access token from the response
access_token = response.json().get('access_token')
if not access_token:
    print("Error: Failed to get access token.")
    exit(1)
print(f"Access Token: {access_token}")

# Function to handle rate limiting and pagination for fetching songs
def make_spotify_request(url, headers):
    while True:
        response = requests.get(url, headers=headers)
        
        # Log the response status and content for debugging
        print(f"Requesting URL: {url}")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            return response.json()
        
        elif response.status_code == 429:
            # Rate limit reached, wait for 'Retry-After' seconds before retrying
            retry_after = int(response.headers.get('Retry-After', 5))  # Default to 5 seconds if no header
            print(f"Rate limit hit, waiting for {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            print(f"Error {response.status_code}: {response.text}")
            break

# Function to pull random tracks (you can modify the query for more randomness)
def pull_random_songs(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    
    while len(songs) < total_songs:
        # Search for random songs using a broad search query
        search_url = f"https://api.spotify.com/v1/search?q=a&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs

    return songs

# Pull 10,000 random songs
#random_songs = pull_random_songs(10000)

# Convert to DataFrame for easier manipulation
#if random_songs:
 #   songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
  #  print(songs_df.head())

    # Optionally, save the data to a CSV file
   # songs_df.to_csv('random_songs.csv', index=False)
#else:
 #   print("No songs were retrieved.")

Access Token: BQCxYQBJOfk_LFqYSPSEyqUKdInm25ROISI5N2a6vKb3dt6SIZbvDas5-fmgNjx4FxE5s-3uQrfle39C4g1KfRMRFSEV9Ftk-HOfxhN4XRf1IpjddiG3twxxOCgVjHqQ8CxSLeEw2_A


In [15]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'a'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_a.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: b
                 Track Name                  Artist  \
0                      APT.        ROSÉ, Bruno Mars   
1  All The Stars (with SZA)     Kendrick Lamar, SZA   
2                   Anxiety                 Doechii   
3   ANXIETY (feat. Doechii)  Sleepy Hallow, Doechii   
4               Abracadabra               Lady Gaga   

                                               Album  Popularity  \
0                                               APT.          95   
1  Black Panther The Album Music From And Inspire...          95   
2                                            Anxiety          93   
3                                    Boy Meets World          89   
4                                        Abracadabra          92   

                                           Track URL  
0  https://open.spotify.com/track/5vNRhkKd0yEAg8s...  
1  https://open.spotify.com/track/3GCdLUSnKSMJhs4...  
2  https://open.spotify.com/track/1musbempyJAw5gf...  
3  https://o

In [16]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'b'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_b.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: c
           Track Name                 Artist                 Album  \
0                APT.       ROSÉ, Bruno Mars                  APT.   
1  BIRDS OF A FEATHER          Billie Eilish  HIT ME HARD AND SOFT   
2    Die With A Smile  Lady Gaga, Bruno Mars      Die With A Smile   
3     Champagne Coast           Blood Orange       Coastal Grooves   
4   BAILE INoLVIDABLE              Bad Bunny  DeBÍ TiRAR MáS FOToS   

   Popularity                                          Track URL  
0          95  https://open.spotify.com/track/5vNRhkKd0yEAg8s...  
1          99  https://open.spotify.com/track/6dOtVTDdiauQNBQ...  
2         100  https://open.spotify.com/track/2plbrEY59IikOBg...  
3          79  https://open.spotify.com/track/2KufM8PiQY4i52X...  
4          97  https://open.spotify.com/track/2lTm559tuIvatlT...  


In [17]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'c'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_c.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: d
                                Track Name  \
0                           Pink Pony Club   
1  Clean Baby Sleep White Noise (Loopable)   
2                               HOT TO GO!   
3                                The Giver   
4                  Cowgirls (feat. ERNEST)   

                                              Artist  \
0                                      Chappell Roan   
1  Dream Supplier, Baby Sleeps, Background White ...   
2                                      Chappell Roan   
3                                      Chappell Roan   
4                              Morgan Wallen, ERNEST   

                                     Album  Popularity  \
0  The Rise and Fall of a Midwest Princess          89   
1       Best White Noise For Sleeping Baby          98   
2  The Rise and Fall of a Midwest Princess          83   
3                                The Giver          89   
4                      One Thing At A Time          85   

         

In [14]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'd'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_d.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: e
          Track Name                 Artist                       Album  \
0              NOKIA                  Drake         $ome $exy $ongs 4 U   
1   Die With A Smile  Lady Gaga, Bruno Mars            Die With A Smile   
2               APT.       ROSÉ, Bruno Mars                        APT.   
3  DENIAL IS A RIVER                Doechii  Alligator Bites Never Heal   
4      Dark Thoughts              Lil Tecca               Dark Thoughts   

   Popularity                                          Track URL  
0          93  https://open.spotify.com/track/2u9S9JJ6hTZS3Vf...  
1         100  https://open.spotify.com/track/2plbrEY59IikOBg...  
2          95  https://open.spotify.com/track/5vNRhkKd0yEAg8s...  
3          92  https://open.spotify.com/track/1eTaznNW4Xxtx9z...  
4          91  https://open.spotify.com/track/7EW7Yivb93qKAtp...  


In [26]:
CLIENT_ID = '3c5805d570ac43279a5585c95db31a11'
CLIENT_SECRET = '1248b4671b284effbe92c559c4f51bec'

# Base64 encode the client_id and client_secret
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode("utf-8")

# Define the URL for Spotify's token endpoint
TOKEN_URL = "https://accounts.spotify.com/api/token"

# Prepare the data for the POST request
headers = {
    "Authorization": f"Basic {auth_header}"
}

data = {
    "grant_type": "client_credentials"
}

# Make the POST request to get the token
response = requests.post(TOKEN_URL, headers=headers, data=data)

# Get the access token from the response
access_token = response.json().get('access_token')
if not access_token:
    print("Error: Failed to get access token.")
    exit(1)
print(f"Access Token: {access_token}")


def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'e'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_e.csv', index=False)
else:
    print("No songs were retrieved.")

Access Token: BQBIBGH2xsjnXBSHd2T41JJfOAvV23HEf3_jUy8ujppdBUzhsjfcTt_NbcBD4jtHJ7t__m4qD1wbTqRp0FnhC9whu3hsaCOYnsAE9rDXaFMY91TD37KtDhLQEOMydLktH5aEo5zGbmU
Switching to search term: f
              Track Name                      Artist                  Album  \
0                    EoO                   Bad Bunny   DeBÍ TiRAR MáS FOToS   
1            EVIL J0RDAN               Playboi Carti                  MUSIC   
2               Espresso           Sabrina Carpenter               Espresso   
3               euphoria              Kendrick Lamar               euphoria   
4  El Mayor de los Ranas  Victor Valverde, JR Torres  El Mayor de los Ranas   

   Popularity                                          Track URL  
0          93  https://open.spotify.com/track/6J5kc12BW5HuP3d...  
1          92  https://open.spotify.com/track/6iycYUk3oB0NPMd...  
2          89  https://open.spotify.com/track/2qSkIjg1o9h3YT9...  
3          82  https://open.spotify.com/track/77DRzu7ERs0TX3r...  
4       

In [27]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'f'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_f.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: g
                    Track Name                       Artist  \
0  Feel It - From “Invincible”                         d4vd   
1                    Free Bird               Lynyrd Skynyrd   
2   FE!N (feat. Playboi Carti)  Travis Scott, Playboi Carti   
3                        FOMDJ                Playboi Carti   
4              Fat Juicy & Wet        Sexyy Red, Bruno Mars   

                               Album  Popularity  \
0        Feel It (From “Invincible”)          90   
1  Pronounced' Leh-'Nerd 'Skin-'Nerd          80   
2                             UTOPIA          88   
3            MUSIC - SORRY 4 DA WAIT          81   
4                    Fat Juicy & Wet          83   

                                           Track URL  
0  https://open.spotify.com/track/3SAga35lAPYdjj3...  
1  https://open.spotify.com/track/5EWPGh7jbTNO2wa...  
2  https://open.spotify.com/track/42VsgItocQwOQC3...  
3  https://open.spotify.com/track/7bO7a1KrtmGNDIO...  
4  ht

In [28]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'g'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_g.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: h
                      Track Name                     Artist  \
0                      Get Jiggy        B Jack$, ZEDDY WILL   
1               Good Luck, Babe!              Chappell Roan   
2                      Gata Only        FloyyMenor, Cris Mj   
3  Guess featuring billie eilish  Charli xcx, Billie Eilish   
4                      The Giver              Chappell Roan   

                                               Album  Popularity  \
0                                          Get Jiggy          79   
1                                   Good Luck, Babe!          95   
2                                          Gata Only          89   
3  Brat and it’s completely different but also st...          86   
4                                          The Giver          89   

                                           Track URL  
0  https://open.spotify.com/track/55NuTxRk1TyKzpg...  
1  https://open.spotify.com/track/0WbMK4wrZ1wFSty...  
2  https://open.spo

In [29]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'h'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_h.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: i
       Track Name          Artist                                    Album  \
0        Headlock     Imogen Heap      Speak for Yourself (Deluxe Version)   
1      Hard Times        Paramore                           After Laughter   
2      HOT TO GO!   Chappell Roan  The Rise and Fall of a Midwest Princess   
3         HUMBLE.  Kendrick Lamar                                    DAMN.   
4  Hometown Glory           Adele                                       19   

   Popularity                                          Track URL  
0          90  https://open.spotify.com/track/63Pi2NAx5yCgeLh...  
1          84  https://open.spotify.com/track/0w5Bdu51Ka25Pf3...  
2          83  https://open.spotify.com/track/4xdBrk0nFZaP54v...  
3          89  https://open.spotify.com/track/7KXjTSCq5nL1LoY...  
4          73  https://open.spotify.com/track/19QzuPTNZHJ2SEu...  


In [30]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'i'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_i.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: j
                                      Track Name                       Artist  \
0                               Die With A Smile        Lady Gaga, Bruno Mars   
1                                I'm The Problem                Morgan Wallen   
2                      Indigo (feat. Avery Anna)       Sam Barber, Avery Anna   
3                                           Iris            The Goo Goo Dolls   
4  I Remember Everything (feat. Kacey Musgraves)  Zach Bryan, Kacey Musgraves   

               Album  Popularity  \
0   Die With A Smile         100   
1    I'm The Problem          87   
2      Restless Mind          87   
3  Dizzy up the Girl          91   
4         Zach Bryan          88   

                                           Track URL  
0  https://open.spotify.com/track/2plbrEY59IikOBg...  
1  https://open.spotify.com/track/3HlWVKwU0JqJXqJ...  
2  https://open.spotify.com/track/1KZG1H1OCugGqIR...  
3  https://open.spotify.com/track/6Qyc6fS4DsZjB2

In [31]:
# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'j'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_1000_j.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: k
                 Track Name                 Artist                     Album  \
0                      Juno      Sabrina Carpenter            Short n' Sweet   
1              Just In Case          Morgan Wallen              Just In Case   
2  Just Us (feat. Doja Cat)  Jack Harlow, Doja Cat  Just Us (feat. Doja Cat)   
3      Just the Way You Are             Bruno Mars      Doo-Wops & Hooligans   
4                     Judas              Lady Gaga             Born This Way   

   Popularity                                          Track URL  
0          88  https://open.spotify.com/track/21B4gaTWnTkuSh7...  
1          86  https://open.spotify.com/track/02Ya2Q08bBKFqUJ...  
2          82  https://open.spotify.com/track/1SVhTS4l0sLFsP2...  
3          88  https://open.spotify.com/track/7BqBn9nzAq8spo5...  
4          75  https://open.spotify.com/track/0QkWikH5Z3U0f79...  


In [4]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'k'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_k.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: l
                 Track Name                        Artist  \
0                      Khé?  Rauw Alejandro, Romeo Santos   
1  All The Stars (with SZA)           Kendrick Lamar, SZA   
2                   Kiss Me      Sixpence None The Richer   
3              Karma Police                     Radiohead   
4                 KLOuFRENS                     Bad Bunny   

                                               Album  Popularity  \
0                                       Cosa Nuestra          89   
1  Black Panther The Album Music From And Inspire...          95   
2                           Sixpence None the Richer          86   
3                                        OK Computer          83   
4                               DeBÍ TiRAR MáS FOToS          90   

                                           Track URL  
0  https://open.spotify.com/track/1YYfyJ2yCvGFCxw...  
1  https://open.spotify.com/track/3GCdLUSnKSMJhs4...  
2  https://open.spotify.com/tra

In [5]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'l'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_l.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: m
                 Track Name               Artist  \
0         luther (with sza)  Kendrick Lamar, SZA   
1         luther (with sza)  Kendrick Lamar, SZA   
2  All The Stars (with SZA)  Kendrick Lamar, SZA   
3                      Loco           Neton Vega   
4             Te Quería Ver   Alemán, Neton Vega   

                                               Album  Popularity  \
0                                                GNX          93   
1                                                GNX          94   
2  Black Panther The Album Music From And Inspire...          95   
3                                               Loco          88   
4                                      Te Quería Ver          94   

                                           Track URL  
0  https://open.spotify.com/track/45J4avUb9Ni0bnE...  
1  https://open.spotify.com/track/2CGNAOSuO1MEFCb...  
2  https://open.spotify.com/track/3GCdLUSnKSMJhs4...  
3  https://open.spotify.com/tr

In [6]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'm'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_m.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: n
                     Track Name                         Artist  \
0  tv off (feat. lefty gunplay)  Kendrick Lamar, Lefty Gunplay   
1           MILLION DOLLAR BABY                  Tommy Richman   
2                         Messy                     Lola Young   
3                          MUTT                    Leon Thomas   
4                   Money Trees       Kendrick Lamar, Jay Rock   

                    Album  Popularity  \
0                     GNX          94   
1     MILLION DOLLAR BABY          88   
2                   Messy          92   
3                    MUTT          80   
4  good kid, m.A.A.d city          84   

                                           Track URL  
0  https://open.spotify.com/track/0aB0v4027ukVziU...  
1  https://open.spotify.com/track/5AJ9hqTS2wcFQCE...  
2  https://open.spotify.com/track/3SKH53SPQbEnZR4...  
3  https://open.spotify.com/track/1mh9eHVRdNhzryG...  
4  https://open.spotify.com/track/2HbKqm4o0w5wEeE... 

In [7]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'n'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_n.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: o
       Track Name           Artist                Album  Popularity  \
0           NOKIA            Drake  $ome $exy $ongs 4 U          94   
1     Not Like Us   Kendrick Lamar          Not Like Us          96   
2           NOKIA            Drake  $ome $exy $ongs 4 U          65   
3  No One Noticed       The Marías            Submarine          92   
4        Nutshell  Alice In Chains         Jar Of Flies          79   

                                           Track URL  
0  https://open.spotify.com/track/2u9S9JJ6hTZS3Vf...  
1  https://open.spotify.com/track/6AI3ezQ4o3HUoP6...  
2  https://open.spotify.com/track/0G39UVgcBxSEczg...  
3  https://open.spotify.com/track/3siwsiaEoU4Kuuc...  
4  https://open.spotify.com/track/2JuasWPUodaUxf5...  


In [8]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'o'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_o.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: p
                                          Track Name  \
0                                           Ordinary   
1                     Ojos Tristes (with The Marías)   
2                         Ordinary - Wedding Version   
3     One Of The Girls (with JENNIE, Lily Rose Depp)   
4  Or Nah (feat. The Weeknd, Wiz Khalifa & DJ Mus...   

                                            Artist  \
0                                      Alex Warren   
1           Selena Gomez, benny blanco, The Marías   
2                                      Alex Warren   
3               The Weeknd, JENNIE, Lily-Rose Depp   
4  Ty Dolla $ign, The Weeknd, Wiz Khalifa, Mustard   

                                               Album  Popularity  \
0                                           Ordinary          95   
1                            I Said I Love You First          84   
2                         Ordinary (Wedding Version)          80   
3  The Idol Episode 4 (Music from the 

In [9]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'p'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_p.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: q
                    Track Name          Artist  \
0               Pink Pony Club   Chappell Roan   
1                   Pink Skies      Zach Bryan   
2                    party 4 u      Charli xcx   
3  Pushing It Down and Praying  Lizzy McAlpine   
4                 Pink + White     Frank Ocean   

                                     Album  Popularity  \
0  The Rise and Fall of a Midwest Princess          89   
1                               Pink Skies          87   
2                      how i'm feeling now          83   
3              Pushing It Down and Praying          69   
4                                   Blonde          89   

                                           Track URL  
0  https://open.spotify.com/track/1k2pQc5i348DCHw...  
1  https://open.spotify.com/track/4ZJ4vzLQekI0Wnt...  
2  https://open.spotify.com/track/2RdEC8Ff83WkX7k...  
3  https://open.spotify.com/track/4wO2JlPZw72Xlu6...  
4  https://open.spotify.com/track/3xKsf9qdS1Cyv

In [10]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'q'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_q.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: r
                      Track Name                                Artist  \
0                 Qué Pasaría...             Rauw Alejandro, Bad Bunny   
1                 Qué Pasaría...             Rauw Alejandro, Bad Bunny   
2  Q U E V A S H A C E R H O Y ?               Omar Courtz, De La Rose   
3                       Que Onda  Calle 24, Chino Pacas, Fuerza Regida   
4                          QLONA                   KAROL G, Peso Pluma   

                                 Album  Popularity  \
0                       Qué Pasaría...          75   
1                         Cosa Nuestra          91   
2                         PRIMERA MUSA          87   
3                             Que Onda          79   
4  MAÑANA SERÁ BONITO (BICHOTA SEASON)          83   

                                           Track URL  
0  https://open.spotify.com/track/0OxKf18dUo2G5dF...  
1  https://open.spotify.com/track/0l0vcZMU7AOeQmU...  
2  https://open.spotify.com/track/0Sr

In [11]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'r'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_r.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: s
                     Track Name                     Artist  \
0                Revolving door                 Tate McRae   
1  RATHER LIE (with The Weeknd)  Playboi Carti, The Weeknd   
2                       Runaway        Kanye West, Pusha T   
3                  Rattlin' Bog             Carlyle Fraser   
4                          Ribs                      Lorde   

                               Album  Popularity  \
0                   So Close To What          91   
1                              MUSIC          92   
2  My Beautiful Dark Twisted Fantasy          84   
3                        Pint of Ale          53   
4                       Pure Heroine          82   

                                           Track URL  
0  https://open.spotify.com/track/541sN2qNfIlllGn...  
1  https://open.spotify.com/track/68qeaZhtMZ6abrJ...  
2  https://open.spotify.com/track/3DK6m7It6Pw857F...  
3  https://open.spotify.com/track/2VjrFvthQjw7BS8...  
4  https://

Agreggate the Data Frames

In [16]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 's'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_s.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: t
                 Track Name               Artist  \
0         luther (with sza)  Kendrick Lamar, SZA   
1               Sailor Song           Gigi Perez   
2  All The Stars (with SZA)  Kendrick Lamar, SZA   
3                   Tweaker                 GELO   
4               squabble up       Kendrick Lamar   

                                               Album  Popularity  \
0                                                GNX          93   
1                                        Sailor Song          96   
2  Black Panther The Album Music From And Inspire...          95   
3                                            Tweaker          82   
4                                                GNX          90   

                                           Track URL  
0  https://open.spotify.com/track/45J4avUb9Ni0bnE...  
1  https://open.spotify.com/track/2262bWmqomIaJXw...  
2  https://open.spotify.com/track/3GCdLUSnKSMJhs4...  
3  https://open.spotify.com/tr

In [17]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 't'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_t.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: u
                                       Track Name  \
0                                     Not Like Us   
1                    tv off (feat. lefty gunplay)   
2  Sticky (feat. GloRilla, Sexyy Red & Lil Wayne)   
3                     Like Him (feat. Lola Young)   
4                   Timeless (feat Playboi Carti)   

                                              Artist              Album  \
0                                     Kendrick Lamar        Not Like Us   
1                      Kendrick Lamar, Lefty Gunplay                GNX   
2  Tyler, The Creator, GloRilla, Sexyy Red, Lil W...        CHROMAKOPIA   
3                     Tyler, The Creator, Lola Young        CHROMAKOPIA   
4                          The Weeknd, Playboi Carti  Hurry Up Tomorrow   

   Popularity                                          Track URL  
0          96  https://open.spotify.com/track/6AI3ezQ4o3HUoP6...  
1          94  https://open.spotify.com/track/0aB0v4027ukVziU...  
2

In [20]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'u'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_u.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: v
         Track Name                    Artist                         Album  \
0         undressed                     sombr                     undressed   
1         Unwritten       Natasha Bedingfield                     Unwritten   
2  Under Your Spell            Snow Strippers               April Mixtape 3   
3     Unforgettable  French Montana, Swae Lee                  Jungle Rules   
4          Umbrella            Rihanna, JAY-Z  Good Girl Gone Bad: Reloaded   

   Popularity                                          Track URL  
0          87  https://open.spotify.com/track/4AajxCEwGEsmHmT...  
1          89  https://open.spotify.com/track/3U5JVgI2x4rDyHG...  
2          81  https://open.spotify.com/track/3iRoXGFrF8O0jTu...  
3          88  https://open.spotify.com/track/3B54sVLJ402zGa6...  
4          85  https://open.spotify.com/track/49FYlytm3dAAraY...  


In [21]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'v'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_v.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: w
             Track Name          Artist  \
0                Vienna      Billy Joel   
1  VOY A LLeVARTE PA PR       Bad Bunny   
2             Von dutch      Charli xcx   
3          Viva La Vida        Coldplay   
4               vampire  Olivia Rodrigo   

                                       Album  Popularity  \
0              The Stranger (Legacy Edition)          84   
1                       DeBÍ TiRAR MáS FOToS          94   
2                                       BRAT          82   
3  Viva La Vida or Death and All His Friends          89   
4                                       GUTS          87   

                                           Track URL  
0  https://open.spotify.com/track/4U45aEWtQhrm8A5...  
1  https://open.spotify.com/track/59D4DOkspUbWyMm...  
2  https://open.spotify.com/track/3Y1EvIgEVw51Xtg...  
3  https://open.spotify.com/track/1mea3bSkSGXuIRv...  
4  https://open.spotify.com/track/1kuGVB7EU95pJOb...  


In [22]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'w'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_w.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: x
   Track Name         Artist                 Album  Popularity  \
0       NOKIA          Drake   $ome $exy $ongs 4 U          94   
1  WILDFLOWER  Billie Eilish  HIT ME HARD AND SOFT          96   
2     WTHELLY          Rob49               WTHELLY          68   
3   Worst Way    Riley Green          Way Out Here          84   
4  We Hug Now    Sydney Rose            We Hug Now          86   

                                           Track URL  
0  https://open.spotify.com/track/2u9S9JJ6hTZS3Vf...  
1  https://open.spotify.com/track/3QaPy1KgI7nu9FJ...  
2  https://open.spotify.com/track/03gyVoP8A8A3mdc...  
3  https://open.spotify.com/track/1osfLqL6L2iQsir...  
4  https://open.spotify.com/track/7zFio8WT0tTBqLs...  


In [23]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'x'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_x.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: y
                         Track Name                              Artist  \
0                     XO Tour Llif3                        Lil Uzi Vert   
1              X Gon' Give It To Ya                                 DMX   
2  X (feat. Maluma & Ozuna) - Remix  Nicky Jam, J Balvin, Maluma, Ozuna   
3                    XXX. FEAT. U2.                  Kendrick Lamar, U2   
4                            Xposed                          Ken Carson   

                                     Album  Popularity  \
0                            Luv Is Rage 2          81   
1  The Definition Of X: Pick Of The Litter          73   
2         X (feat. Maluma & Ozuna) [Remix]          85   
3                                    DAMN.          72   
4                               More Chaos          66   

                                           Track URL  
0  https://open.spotify.com/track/7GX5flRQZVHRAGd...  
1  https://open.spotify.com/track/1zzxoZVylsna2BQ...  
2  http

In [24]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'y'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_y.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: z
                  Track Name            Artist              Album  Popularity  \
0          YOUR WAY'S BETTER     Forrest Frank  YOUR WAY'S BETTER          66   
1  You Should Probably Leave   Chris Stapleton      Starting Over          85   
2                     Yellow          Coldplay         Parachutes          92   
3                        You  Lloyd, Lil Wayne        Street Love          82   
4                   Y.M.C.A.    Village People           Cruisin'          67   

                                           Track URL  
0  https://open.spotify.com/track/4qrbjoNP9NZwe79...  
1  https://open.spotify.com/track/2UikqkwBv7aIvli...  
2  https://open.spotify.com/track/3AJwUDP919kvQ9Q...  
3  https://open.spotify.com/track/5Y77SQxEr1eiofP...  
4  https://open.spotify.com/track/4YOJFyjqh8eAcbK...  


In [25]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = 'z'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_z.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: a
                           Track Name                             Artist  \
0                              Zombie                    The Cranberries   
1  ZEZE (feat. Travis Scott & Offset)  Kodak Black, Offset, Travis Scott   
2                           Zombieboy                          Lady Gaga   
3                              Zombie                         Bad Wolves   
4                         I Never Lie                           Zach Top   

                                               Album  Popularity  \
0  No Need To Argue (The Complete Sessions 1994-1...          86   
1                                      Dying to Live          79   
2                                             MAYHEM          82   
3                                            Disobey          77   
4                          Cold Beer & Country Music          85   

                                           Track URL  
0  https://open.spotify.com/track/7EZC6E7UjZe63f1..

In [30]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = '2'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_2.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: 3
  Track Name         Artist                         Album  Popularity  \
0        2AM    BigXthaPlug                     TAKE CARE          75   
1         28     Zach Bryan  The Great American Bar Scene          82   
2     20 Min   Lil Uzi Vert        Luv Is Rage 2 (Deluxe)          84   
3       2024  Playboi Carti       MUSIC - SORRY 4 DA WAIT          81   
4  24K Magic     Bruno Mars                     24K Magic          85   

                                           Track URL  
0  https://open.spotify.com/track/26TX1oZbaDeKxfT...  
1  https://open.spotify.com/track/5iJKGpnFfvbjZJe...  
2  https://open.spotify.com/track/0uxSUdBrJy9Un0E...  
3  https://open.spotify.com/track/1bAv1DWUibxCIjA...  
4  https://open.spotify.com/track/6b8Be6ljOzmkOmF...  


In [32]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = '8'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_8.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: 9
         Track Name         Artist  \
0               865  Morgan Wallen   
1             8TEEN         Khalid   
2  8am in Charlotte          Drake   
3  867-5309 / Jenny   Tommy Tutone   
4            8 Mile         Eminem   

                                               Album  Popularity  \
0                        Dangerous: The Double Album          71   
1                                      American Teen          76   
2                                   For All The Dogs          70   
3                                   Tommy Tutone - 2          64   
4  8 Mile (Music From And Inspired By The Motion ...          64   

                                           Track URL  
0  https://open.spotify.com/track/0sKEilrw1GrBlG6...  
1  https://open.spotify.com/track/5bgwqaRSS3M8WHW...  
2  https://open.spotify.com/track/0w3Mfe4PIVjuFjJ...  
3  https://open.spotify.com/track/6iX1f3r7oUJnMbG...  
4  https://open.spotify.com/track/5NmL2zxnlt9GiTB...  


In [35]:
import time
import requests

# Function to handle Spotify API request
def make_spotify_request(url, headers):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    return response.json()

# Function to pull songs one at a time, based on different search terms
def pull_random_songs_v2(total_songs=10000):
    songs = []
    offset = 0  # Starting point for pagination
    limit = 50  # Max number of songs per request
    search_term = '1'  # Starting search term
    
    # We will fetch songs one search term at a time (e.g., 'a', 'b', 'c', etc.)
    while len(songs) < total_songs:
        # Construct the search URL
        search_url = f"https://api.spotify.com/v1/search?q={search_term}&type=track&limit={limit}&offset={offset}"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        
        # Make the API request and get data
        data = make_spotify_request(search_url, headers)
        
        # Check if data is None or if 'tracks' is missing in the response
        if not data or 'tracks' not in data:
            print("Error: Invalid data returned.")
            break
        
        # Extract track data
        tracks = data['tracks']['items']
        
        # Append track info to the songs list
        for track in tracks:
            song_info = {
                "Track Name": track['name'],
                "Artist": ", ".join([artist['name'] for artist in track['artists']]),
                "Album": track['album']['name'],
                "Popularity": track['popularity'],
                "Track URL": track['external_urls']['spotify']
            }
            songs.append(song_info)
        
        # If we've pulled enough songs, break the loop
        if len(songs) >= total_songs:
            break
        
        # Increment the offset for the next page of results
        offset += limit
        
        # Pause to handle rate limiting
        time.sleep(1)  # Adjust the sleep time based on your rate limit needs
        
        # If we hit the 1000-song limit, move to the next search term
        if offset >= 1000:
            offset = 0  # Reset offset for the new search term
            # Get the next search term (from 'a' to 'z', then cycle back)
            search_term = chr(ord(search_term) + 1)
            if search_term > 'z':  # Reset back to 'a' after 'z'
                search_term = 'a'
            print(f"Switching to search term: {search_term}")
    
    return songs

# Pull 10,000 random songs one search at a time
random_songs = pull_random_songs_v2(total_songs=1000)

# Convert to DataFrame for easier manipulation
if random_songs:
    import pandas as pd
    songs_df = pd.DataFrame(random_songs)
    # Show the first few rows of the DataFrame
    print(songs_df.head())

    # Optionally, save the data to a CSV file
    songs_df.to_csv('random_songs_10000_1.csv', index=False)
else:
    print("No songs were retrieved.")

Switching to search term: 2
               Track Name                       Artist  \
0              15 Minutes            Sabrina Carpenter   
1                    1 AM                          Civ   
2          1-800-273-8255  Logic, Alessia Cara, Khalid   
3                      16                    Baby Keem   
4  1979 - Remastered 2012        The Smashing Pumpkins   

                                               Album  Popularity  \
0                            Short n' Sweet (Deluxe)          86   
1                                               1 AM          78   
2                                          Everybody          76   
3                                   The Melodic Blue          81   
4  Mellon Collie And The Infinite Sadness (Deluxe...          80   

                                           Track URL  
0  https://open.spotify.com/track/1zhvxTuSha22nsU...  
1  https://open.spotify.com/track/6rL8u0RmBmL1Mp5...  
2  https://open.spotify.com/track/5tz69p7tJuGPeMG

In [36]:
import pandas as pd
songs = ["random_songs_10000_a.csv", "random_songs_1000_b.csv", "random_songs_1000_c.csv", "random_songs_1000_d.csv", "random_songs_1000_e.csv", "random_songs_1000_f.csv", "random_songs_1000_g.csv","random_songs_1000_h.csv", "random_songs_1000_i.csv", "random_songs_1000_j.csv", "random_songs_10000_k.csv", "random_songs_10000_l.csv", "random_songs_10000_m.csv", "random_songs_10000_n.csv", "random_songs_10000_o.csv", "random_songs_10000_p.csv", "random_songs_10000_q.csv","random_songs_10000_r.csv", "random_songs_10000_s.csv", "random_songs_10000_t.csv","random_songs_10000_u.csv", "random_songs_10000_v.csv", "random_songs_10000_w.csv", "random_songs_10000_x.csv", "random_songs_10000_y.csv", "random_songs_10000_z.csv", "random_songs_10000_2.csv", "random_songs_10000_1.csv", "random_songs_10000_8.csv"]


combined = []

for i, song in enumerate(songs):
    read = pd.read_csv(song)
    
    read["song"] = f"Song {i}"
    
    combined.append(read)
    


random_songs_10000 = pd.concat(combined)

In [37]:
random_songs_10000.to_csv("SONGS.csv", index = True)

In [38]:
no_duplicates=random_songs_10000.drop_duplicates(subset=['Track Name', 'Artist'])
no_duplicates

,Track Name,Artist,Album,Popularity,Track URL,song
0,APT.,"ROSÉ, Bruno Mars",APT.,95,https://open.spotify.com/track/5vNRhkKd0yEAg8s...,Song 0
1,All The Stars (with SZA),"Kendrick Lamar, SZA",Black Panther The Album Music From And Inspire...,95,https://open.spotify.com/track/3GCdLUSnKSMJhs4...,Song 0
2,Anxiety,Doechii,Anxiety,93,https://open.spotify.com/track/1musbempyJAw5gf...,Song 0
3,ANXIETY (feat. Doechii),"Sleepy Hallow, Doechii",Boy Meets World,89,https://open.spotify.com/track/3FjK86616FbluOf...,Song 0
4,Abracadabra,Lady Gaga,Abracadabra,92,https://open.spotify.com/track/5ZLUm9eab8y3tqQ...,Song 0
...,...,...,...,...,...,...
1034,99 (with Future),"Lil Baby, Future",WHAM (Extended Version),63,https://open.spotify.com/track/3wD05dVOHFfqr3K...,Song 28
1036,976-EVIL,Deftones,Diamond Eyes,65,https://open.spotify.com/track/3hb4W4FUPjyWUbY...,Song 28
1037,"9 Préludes, Op. 103: No. 3 in G Minor","Gabriel Fauré, Lucas Debargue","""Cantique"" - Deep Concentration & Focus",0,https://open.spotify.com/track/2zcpkmkWYckG7X6...,Song 28
1038,90mph,"JBEE, Sillage",90mph,70,https://open.spotify.com/track/6uT2TsDrCrXue7R...,Song 28


In [39]:
no_duplicates.to_csv("SONGS.csv", index = True)